In [ ]:
using MagneticLaplacianSparsifier
using Graphs, MetaGraphs, Random, IterTools, LinearAlgebra, GraphPlot, Arpack, Plots, SparseArrays

In [ ]:
# graph generation
q = 0;
n = 10;
eta = 0.5;
p = 0.9;

rng = Random.default_rng();
comp_graph = gen_graph_ero(rng, n, p, eta);

# sample a CRSF
crsf = multi_type_spanning_forest(rng, comp_graph, q)

# get the nodes in the cycle(s)
cycles = get_prop(crsf,:cycle_nodes)
# get the branches in the (reverse) order there were sampled
branches = get_prop(crsf,:branches)

flt_branches = collect(Iterators.flatten(branches))
flt_cycles = collect(Iterators.flatten(cycles))

print("peeling leafs order: ", flt_branches, "\n")
print("cycle nodes: ",flt_cycles)

# plotting
nodelabel = 1:nv(crsf)
gplot(crsf, nodelabel=nodelabel)

In [ ]:
# indices array putting the nodes in the right order
ind = [flt_branches; setdiff(1:n,flt_branches)];
# construct Laplacian
B = sp_magnetic_incidence(crsf; oriented=true);
L = B*B';
# reorder Laplacian
L_perm = L[ind,ind];
# find cholesky
lower_factor = cholesky(L, perm = ind).L;

chol_matrix = sparse(lower_factor)
# number of diag entries
print("choleski of L with custom perm: ", nnz(chol_matrix) - n, " off diagonal entries \n")

# upper bound on number of non-zero entries (Prop 2 in paper)
# n + sum_{cycle c}(n_c - 3)
bound = 0;
for i=1:length(flt_cycles)
    bound += length(flt_cycles[i]) - 3
end
bound  += n;
print("theoretical upper bound: ", bound, " off diagonal entries \n")
#print(sparse(lower_factor))

# cholesky of random permuted Laplacian
p = randperm(n);
lower_factor_rand_perm = sparse(cholesky(L, perm = p).L);
print("choleski of L with RANDOM permutation: ", nnz(lower_factor_rand_perm) - n, " off diagonal entries \n")
#print(lower_factor_rand_perm)

# cholesky of raw Laplacian
lower_factor_raw = sparse(cholesky(L).L);
print("choleski of L with AUTOMATIC permutation: ", nnz(lower_factor_raw) - n, " off diagonal entries \n")
